In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf

In [3]:
data = pf.getdata("/Users/zhangchuanyu/sky_maps_new_64_v6.fits")

In [4]:
EBV = data['SFD']
HI = data['HI']/1e21
conversion_factor = 2*1e20/1e21 ### A factor to convert CO to H2
H2 = data['CO10']*conversion_factor
cov_1 = np.zeros(len(EBV))

In [11]:
def correlation_spearman(v_1,v_2):
    
    temp_v_1 = v_1.argsort()
    v_1_rank = temp_v_1.argsort()
    temp_v_2 = v_2.argsort()
    v_2_rank = temp_v_2.argsort()
    cov_1 = np.zeros(len(v_2))

    for x in range(len(v_2)):
        v_1_i = v_1_rank[x]
        v_2_i = v_2_rank[x]
        cov_1[x]=(v_1_i-np.mean(v_1_rank))*(v_2_i-np.mean(v_2_rank))
    mean_std = np.mean(cov_1)
    std_v_1 = np.std(v_1_rank)
    std_v_2 = np.std(v_2_rank)

    return mean_std/(std_v_1*std_v_2)   

def correlation_pearson(v_1,v_2):
    cov_1 = np.zeros(len(v_1))
    for i in range(len(v_1)):
        cov_1[i] = (v_1[i]-np.mean(v_1))*(v_2[i]-np.mean(v_2))
    mean_std = np.mean(cov_1)
    std_v_1 = np.std(v_1)
    std_v_2 = np.std(v_2)
    
    return mean_std/(std_v_1*std_v_2)

def correlation_spearman_error(v_1,v_2):
    boots_time = 200
    cor_error = []
    for i in range(boots_time):
        random_index = np.random.randint(0,len(v_1),len(v_1))
        cor_error.append(correlation_spearman(v_1[random_index],v_2[random_index]))
    return np.std(cor_error)

def correlation_pearson_error(v_1,v_2):
    boots_time = 200
    cor_error = []
    for i in range(boots_time):
        random_index = np.random.randint(0,len(v_1),len(v_1))
        cor_error.append(correlation_pearson(v_1[random_index],v_2[random_index]))
    return np.std(cor_error)

In [12]:
print("Spearman correlation is %0.3f +- %0.4f" %(correlation_spearman(EBV,HI),correlation_spearman_error(EBV,HI)))
print("Pearson correlation is %0.3f +- %0.4f" %(correlation_pearson(EBV,HI),correlation_pearson_error(EBV,HI)))

Spearman correlation is 0.976 +- 0.0003
Pearson correlation is 0.767 +- 0.0169


In [14]:
print("Spearman correlation is %0.3f +- %0.4f" %(correlation_spearman(EBV,H2),correlation_spearman_error(EBV,H2)))
print("Pearson correlation is %0.3f +- %0.4f" %(correlation_pearson(EBV,H2),correlation_pearson_error(EBV,H2)))

Spearman correlation is 0.356 +- 0.0047
Pearson correlation is 0.865 +- 0.0142
